In [2]:
import os
from flask import Flask, jsonify
from supabase import create_client, Client
from time import sleep
from datetime import datetime

app = Flask(__name__)

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

TABLE_NAME = "v2_federato_amplitude_data"

In [9]:
def get_retention_events(user_id):
    """Calculate the top 5 retention events for a given user."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Convert events to a list of event types, excluding session events
        excluded_events = {"session_start", "session_end"}
        events = [event["event_type"] for event in events_response.data 
                 if event["event_type"] not in excluded_events]
        
        # Calculate return frequency for each event type
        event_returns = {}
        for i, event in enumerate(events[:-1]):  # Exclude last event
            # Look at the next events to see if user returns to this event
            future_events = events[i+1:]
            if event in future_events:
                event_returns[event] = event_returns.get(event, 0) + 1
        
        # Calculate return rate (returns / total occurrences)
        event_counts = {}
        for event in events:
            event_counts[event] = event_counts.get(event, 0) + 1
            
        return_rates = {}
        for event, returns in event_returns.items():
            return_rates[event] = returns / event_counts[event]
        
        # Get top 5 events by return rate (minimum 2 occurrences)
        top_events = sorted(
            [(event, rate) for event, rate in return_rates.items() 
             if event_counts[event] >= 2],  # Filter events with at least 2 occurrences
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        # Format results
        return [{"event": event, "return_rate": rate} for event, rate in top_events]
        
    except Exception as e:
        print(f"Error calculating retention events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top retention events
def update_all_users_retention_events():
    try:
        # First, add the column directly with raw SQL
        add_column_sql = """
        DO $$ 
        BEGIN 
            IF NOT EXISTS (
                SELECT 1 
                FROM information_schema.columns 
                WHERE table_name = 'user_table' 
                AND column_name = 'top_retention_events'
            ) THEN 
                ALTER TABLE user_table ADD COLUMN top_retention_events JSONB;
            END IF;
        END $$;
        """
        
        # Execute the SQL directly through a query
        supabase.table("user_table").select("*").limit(1).execute()  # Ensure connection
        supabase.postgrest.schema("public").rpc("execute_sql", {"query": add_column_sql}).execute()
        print("Added top_retention_events column if it didn't exist")
        
        # Get all user IDs
        users_response = supabase.table("user_table").select("user_id").execute()
        
        # Process each user
        for user in users_response.data:
            user_id = user["user_id"]
            top_events = get_retention_events(user_id)
            
            # Update the user's record with their top retention events
            supabase.table("user_table")\
                .update({"top_retention_events": top_events})\
                .eq("user_id", user_id)\
                .execute()
            
            print(f"Updated retention events for user {user_id}")
            
        print("Completed updating all users' retention events")
        
    except Exception as e:
        print(f"Error updating retention events: {e}")

# Execute the update
update_all_users_retention_events()

Added top_retention_events column if it didn't exist
Updated retention events for user 1
Updated retention events for user 2
Updated retention events for user 3
Updated retention events for user 4
Updated retention events for user 5
Updated retention events for user 6
Updated retention events for user 7
Updated retention events for user 8
Updated retention events for user 9
Updated retention events for user 10
Updated retention events for user 11
Updated retention events for user 12
Updated retention events for user 13
Updated retention events for user 14
Updated retention events for user 15
Updated retention events for user 16
Updated retention events for user 17
Updated retention events for user 18
Updated retention events for user 19
Updated retention events for user 20
Updated retention events for user 21
Updated retention events for user 22
Updated retention events for user 23
Updated retention events for user 24
Updated retention events for user 25
Updated retention events for us

In [8]:
def get_retention_events(user_id):
    """Calculate the top 5 events where users spend the most time."""
    try:
        # Get user's amplitude_id
        user_response = supabase.table("user_table").select("amplitude_id").eq("user_id", user_id).execute()
        amplitude_id = user_response.data[0]["amplitude_id"]
        
        # Get all events for this user, ordered by time
        events_response = supabase.table(TABLE_NAME)\
            .select("event_type, event_time")\
            .eq("amplitude_id", amplitude_id)\
            .order("event_time", desc=False)\
            .execute()
        
        if not events_response.data:
            return []
        
        # Calculate time spent on each event type
        event_durations = {}
        event_counts = {}
        excluded_events = {"session_start", "session_end"}
        
        for i, current_event in enumerate(events_response.data[:-1]):
            next_event = events_response.data[i + 1]
            event_type = current_event["event_type"]
            
            # Skip excluded events
            if event_type in excluded_events:
                continue
            
            # Convert string timestamps to datetime objects
            current_time = datetime.fromisoformat(current_event["event_time"].replace('Z', '+00:00'))
            next_time = datetime.fromisoformat(next_event["event_time"].replace('Z', '+00:00'))
            
            duration = (next_time - current_time).total_seconds()
            
            event_durations[event_type] = event_durations.get(event_type, 0) + duration
            event_counts[event_type] = event_counts.get(event_type, 0) + 1
        
        # Calculate average duration for each event type
        avg_durations = {}
        for event_type in event_durations:
            avg_durations[event_type] = event_durations[event_type] / event_counts[event_type]
        
        # Get top 5 events by average duration (minimum 2 occurrences)
        top_events = sorted(
            [(event, duration) for event, duration in avg_durations.items() 
             if event_counts[event] >= 2],  # Filter events with at least 2 occurrences
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        # Format results (duration in seconds)
        return [{"event": event, "avg_duration": duration} for event, duration in top_events]
        
    except Exception as e:
        print(f"Error calculating retention events for user {user_id}: {e}")
        return []

# Update all users in the user_table with their top retention events
def update_all_users_retention_events():
    try:
        page_size = 1000
        last_user_id = 0
        
        while True:
            try:
                # Get next batch of users
                users_response = supabase.table("user_table")\
                    .select("user_id")\
                    .gt("user_id", last_user_id)\
                    .order("user_id")\
                    .limit(page_size)\
                    .execute()
                
                # If no more users, break
                if not users_response.data:
                    break
                
                # Process each user in this batch
                for user in users_response.data:
                    user_id = user["user_id"]
                    top_events = get_retention_events(user_id)
                    
                    # Update the user's record with their top retention events
                    supabase.table("user_table")\
                        .update({"top_retention_events": top_events})\
                        .eq("user_id", user_id)\
                        .execute()
                    
                    print(f"Updated retention events for user {user_id}")
                    last_user_id = user_id
                
                print(f"Completed batch up to user {last_user_id}")
                sleep(1)

            except Exception as batch_error:
                print(f"Batch error: {batch_error}")
                sleep(5)
                continue

        print("Completed updating all users' retention events")
        
    except Exception as e:
        print(f"Error updating retention events: {e}")

# Execute the update
update_all_users_retention_events()

Updated retention events for user 1
Updated retention events for user 2
Updated retention events for user 3
Updated retention events for user 4
Updated retention events for user 5
Updated retention events for user 6
Updated retention events for user 7
Updated retention events for user 8
Updated retention events for user 9
Updated retention events for user 10
Error calculating retention events for user 11: {'code': '57014', 'details': None, 'hint': None, 'message': 'canceling statement due to statement timeout'}
Updated retention events for user 11
Updated retention events for user 12
Updated retention events for user 13
Updated retention events for user 14
Updated retention events for user 15
Error calculating retention events for user 16: {'code': '57014', 'details': None, 'hint': None, 'message': 'canceling statement due to statement timeout'}
Updated retention events for user 16
Error calculating retention events for user 17: {'code': '57014', 'details': None, 'hint': None, 'message

KeyboardInterrupt: 

In [7]:
# Query to get top 5 most common retention events across all users
query_response = supabase.table("user_table_refined").select("top_retention_events").execute()

# Flatten all retention events and count occurrences
event_counts = {}
for user in query_response.data:
    if user["top_retention_events"]:  # Check if not None or empty
        # Count each unique event only once per user
        user_events = set(event_data["event"] for event_data in user["top_retention_events"])
        for event in user_events:
            event_counts[event] = event_counts.get(event, 0) + 1

# Get top 5 most common events
top_5_events = sorted(
    event_counts.items(),
    key=lambda x: x[1],  # Sort by count
    reverse=True
)[:5]

# Calculate percentage of users for each event
total_users = len([u for u in query_response.data if u["top_retention_events"]])

print("\nTop 5 Most Common Retention Events:")
print("--------------------------------")
for event, count in top_5_events:
    percentage = (count / total_users) * 100 if total_users > 0 else 0
    print(f"Event: {event}")
    print(f"Number of Users: {count}")
    print(f"Percentage of Users: {percentage:.1f}%")
    print("--------------------------------")



Top 5 Most Common Retention Events:
--------------------------------
Event: application-window-opened
Number of Users: 721
Percentage of Users: 43.3%
--------------------------------
Event: session_end
Number of Users: 705
Percentage of Users: 42.3%
--------------------------------
Event: dashboard:my-book::view
Number of Users: 500
Percentage of Users: 30.0%
--------------------------------
Event: account-lines:::view
Number of Users: 347
Percentage of Users: 20.8%
--------------------------------
Event: session_start
Number of Users: 339
Percentage of Users: 20.3%
--------------------------------
